In [7]:
import pandas as pd
from datetime import datetime
import os

path = "./timestamp_creator"
e4_ts = path+"/input/e4_timestamps.json"  # Columns: sessionId, time

In [8]:
# Only should be one report in json format in this folder
input_data = os.listdir(path+"/input")[0]
input_data
# Only should be one report in json format in this folder
input_data = os.listdir(path+"/input")[0]

In [9]:
sessions = pd.read_json(path+"/input/"+input_data)
sessions = sessions.query('consent == True & close_date != None')
sessions

,_id,close_date,consent,creation_date,current_step,responses,settings,test_id,user
1,6EtIsBQZgh7VLE2Lvn7lu,"Sat, 04 Sep 2021 13:13:45 GMT",True,"Sat, 04 Sep 2021 13:06:52 GMT",83f7ac63-eaac-47d5-a768-5ec5b7c92ac3,"[{'content': {}, 'end_time': 1630757347, 'star...","[{'ambient': False, 'color': [255, 255, 255], ...",6a7a81bf-2d52-457a-bf72-a2701082cbcd,"{'age': '30', 'email': 'andres@gmail.com', 'ge..."


In [10]:
# Load the empatica e4 timestamp sessions
e4_sessions = pd.read_json(e4_ts)
e4_sessions

,sessionId,time
0,6EtIsBQZgh7VLE2Lvn7lu,1630757251


In [11]:
ts = pd.DataFrame()
for i, session in sessions.iterrows():
    responses = session['responses']
    responses = pd.DataFrame.from_dict(responses)
    index = 0
    step_info = {'sessionId': session['_id'],
                 'e4_start': e4_sessions[e4_sessions['sessionId'] == session['_id']].iloc[0]['time']}
    for index, step in responses.iterrows():
        [start, end] = [step['start_time'], step['end_time']]
        dur = end - start
        step_info.update(
            {f'step_{index}': step['step_id'],
             f'type_step_{index}': step['type'],
             f'e4_diff_step_{index}': start - step_info['e4_start'],
             f'start_step_{index}': start,
             f'end_step_{index}': end,
             f'dur_step_{index}': dur})
    ts = ts.append(step_info, ignore_index=True)
ts

,sessionId,e4_start,step_0,type_step_0,e4_diff_step_0,start_step_0,end_step_0,dur_step_0,step_1,type_step_1,...,e4_diff_step_5,start_step_5,end_step_5,dur_step_5,step_6,type_step_6,e4_diff_step_6,start_step_6,end_step_6,dur_step_6
0,6EtIsBQZgh7VLE2Lvn7lu,1.630757e+09,26bcc93d-948e-43e6-8c08-6d603b3c18a9,info,45.0,1.630757e+09,1.630757e+09,51.0,39fb8305-d4ab-4c4c-8915-38a11d055d4d,question,...,286.0,1.630758e+09,1.630758e+09,60.0,33a4b88d-dc42-46d7-a2e4-04d311b7312a,question,346.0,1.630758e+09,1.630758e+09,17.0


In [12]:
# create csv with timestamp
ts.to_csv(path+"/output/timestamp.csv", index=False)